In [1]:
import numpy as np
import pandas as pd

import urllib.request
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier     # obtiene un score (test/pred/submin): 0.3455/0.2211/0.23092
from sklearn.linear_model import LogisticRegression     # obtiene un score (test/pred/submin): 0.277/0.2717/0.27284 ## 
from sklearn.svm import SVC                             # obtiene un score (test/pred/submin): 0.277/0.2717/0.27284 ##  de momento es el mejor modelo con 0'27
from catboost import CatBoostClassifier                 # obtiene un score (test/pred/submin): 0.3623/0.2230/0.23423
from sklearn.ensemble import GradientBoostingClassifier # obtiene un score (test/pred/submin): 0.3661/0.222/
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, RepeatedKFold # RepeatedStratifiedKFold sólo se utiliza para categorías 
from sklearn.cluster import KMeans

In [3]:
# limpieza del dataframe original y codificación de los datos
# definición de las funciones de codificación
def col12_toInt (strin):
    if strin == 'Urgent':
        return 100
    elif strin == 'Trauma':
        return 10
    elif strin == 'Emergency':
        return 1

def col13_toInt (strin):
    if strin == 'Extreme':
        return 100
    elif strin == 'Moderate':
        return 10
    elif strin == 'Minor':
        return 1
        
def col2_toInt (strin):
    if strin == 'a':
        return 44130
    elif strin == 'b':
        return 21667
    elif strin == 'c':
        return 14145
    elif strin == 'd':
        return 6341
    elif strin == 'e':
        return 7678
    elif strin == 'f':
        return 3237
    elif strin == 'g':
        return 1286

def codi_df (df):
    df['2'] = df['2'].apply(lambda x: col2_toInt(x))
    df['12'] = df['12'].apply(lambda x: col12_toInt(x))
    df['13'] = df['13'].apply(lambda x: col13_toInt(x))
    df['15'] = df['15'].apply(lambda x: int(x.replace('-', '')))
    return df 

In [4]:
# Carga de los datos
# carga de los datos de train y test
df_cols = pd.read_csv('columns_meaning.csv')
df_hospital_train = pd.read_csv("hospital_train.csv")

# Carga de los datos de entrega
df_hospital_test = pd.read_csv("hospital_test.csv")
sample = pd.read_csv("sample_submission.csv")

In [5]:
# transformación de los dataframes
# DataFrame de Train y Test
df_hospital_train = df_hospital_train.dropna()      # eliminación de las líneas con Nan
encod_df_hospital_train = df_hospital_train[['2','9','11','12','13','14','15','16']]    # selección de las columnas de train y test
encod_df_hospital_train = codi_df (encod_df_hospital_train)         # codificación de los valores categóricos    

# DataFrame de Entrega
encod_df_hospital_test = df_hospital_test[['2','9','11','12','13','14','15','16']]  # selección de las columnas
encod_df_hospital_test = codi_df(encod_df_hospital_test)    # ejecucción de la función de codificación

# sustitución de los valores nan por valores reales (lo ideal sería hacerlo de forma automática)
encod_df_hospital_test['11'].fillna(8, inplace= True) # 8 es el valor que más se repite tanto en el conjunto de train como en el predicción
encod_df_hospital_test['9'].fillna(2, inplace= True) # 2 es el valor que más se repite tanto en el conjunto de train como en el predicción

     
# conversión de los dataframes a arrays
# generación de los conjuntos de train y test
X = np.array(encod_df_hospital_train)
y = np.array(df_hospital_train['17'])
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle= True, test_size = 0.50, random_state = 42)

# se convierten los datos a vectores
X_pred = np.array(encod_df_hospital_test)
y_sample = sample['days']  

ipykernel_launcher:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:39: SettingWithCopyWarning: 
A value is trying t

In [7]:

voting = KMeans(n_clusters=11, random_state=42).fit(X_train)


In [8]:
# 5. Predecir con el modelo ya entrenado con X_test
predictions = voting.predict(X_test)
predictions

array([3, 0, 0, ..., 7, 5, 0])

In [9]:
y_test

array(['31-40', '21-30', '21-30', ..., '21-30', '21-30', '21-30'],
      dtype=object)

In [98]:
voting.score(X_test, y_test)

0.3691767190609642

In [99]:
# se ejecutan las predicciones con el modelo entrenado
predictions_submit = voting.predict(X_pred)
submission = pd.DataFrame({"id": df_hospital_test['0'], "days": predictions_submit})
voting.score(X_pred, y_sample)

0.22383807871754996

In [156]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")
chequeator(submission)

You're ready to submit!
